In [2]:
import wmfdata as wmf

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [10]:
def read_file(path):
    with open(path, 'r') as f:
        return f.read()

# Analytics Data Lake

Data from the usual data source.

In [ ]:
glob_ret = wmf.query.hive(
    """
    select 
        cohort,
        count(*) as new_eds,
        (sum(if(2nd_month_edits >= 1, 1, 0)) / sum(if(1st_month_edits >= 1, 1, 0))) as ret 
    from neilpquinn.new_editors
    where cohort >= "2014-04"
    group by cohort
    order by cohort asc
    limit 1000
    """
)

glob_ret

In [ ]:
glob_ret.to_csv("data/adl_global_monthly_retention.tsv", sep = "\t", index = False)

In [ ]:
proj_ret = run_hive(
    """
    select 
        wiki,
        cohort,
        count(*) as new_eds,
        (sum(if(2nd_month_edits >= 1, 1, 0)) / sum(if(1st_month_edits >= 1, 1, 0))) as retention
    from neilpquinn.new_editors
    where cohort >= "2017-01"
    group by wiki, cohort
    order by wiki, cohort asc
    limit 20000
    """
)

proj_ret

In [ ]:
proj_ret.to_csv("data/adl_project_monthly_retention.tsv", sep = "\t", index = False)

In [16]:
adl_en_new = wmf.hive.run(
    read_file("queries/adl_enwiki_new_eds.sql").format(
        start = "2016-12",
        end = "2018-02",
        snapshot = "2018-03"
    )
)

adl_en_new.head()

,user,reg_day,mo_1_revs,mo_2_revs,mo_1_reverts,mo_2_reverts
0,!!Hûm Såfår!!,2017-02-23,2,0,1,0
1,!.Jimiey.!,2016-12-17,2,0,2,0
2,!0Clarke,2018-01-06,3,0,1,0
3,!0MinutesADay,2017-11-17,1,0,0,0
4,!23455666A!wee,2017-06-29,2,0,0,0


In [19]:
adl_en_new.to_csv("data/adl_enwiki_new_eds.tsv", sep = "\t", index = False)

# MariaDB application tables
Raw data for comparison

In [ ]:
mdb_en_new_revs = wmf.mariadb.run(
    read_file('queries/mdb_enwiki_new_editor_revisions.sql')
)

mbd_en_new_revs.head()

In [9]:
enwiki_new_revs.to_csv("data/mdb_enwiki_new_ed_revisions.tsv", index = False, sep = "\t")